This project demonstrates a cohort retention rate analysis, an analysis which is common among online businesses. The data is acquired from a [Kaggle competition](https://www.kaggle.com/archit9406/customer-transaction-dataset).

The main steps of the following analysis:
1. Loading data
2. Assign cohorts & calculate monthly offsets
3. Calculate retention rate
4. Visualize data
5. Analyze

In [ ]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

import missingno as msno
from textwrap import wrap

In [ ]:
# Loading dataset

transaction_df = pd.read_excel('transaction.xlsx')

In [ ]:
# View data
transaction_df.head()

In [ ]:
msno.bar(transaction_df)

In [ ]:
# Inspect missing values in the dataset
print(transaction_df.isnull().values.sum())

# Replace the ' 's with NaN
transaction_df = transaction_df.replace(" ",np.NaN)

# Impute the missing values with mean imputation
transaction_df = transaction_df.fillna(transaction_df.mean())

# Count the number of NaNs in the dataset to verify
print(transaction_df.isnull().values.sum())

In [ ]:
print(transaction_df.info())
for col in transaction_df.columns:
    # Check if the column is of object type
    if transaction_df[col].dtypes == 'object':
        # Impute with the most frequent value
        transaction_df[col] = transaction_df[col].fillna(transaction_df[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
print(transaction_df.isnull().values.sum())

In [ ]:
msno.bar(transaction_df)

In [ ]:
transaction_df.info()

Assign cohorts & calculate monthly offsets

In [ ]:
# A function that will parse the date Time based cohort:  1 day of month
def get_month(x): return dt.datetime(x.year, x.month, 1) 

# Create transaction_date column based on month and store in TransactionMonth
transaction_df['TransactionMonth'] = transaction_df['transaction_date'].apply(get_month) 

# Grouping by customer_id and select the InvoiceMonth value
grouping = transaction_df.groupby('customer_id')['TransactionMonth'] 

# Assigning a minimum InvoiceMonth value to the dataset
transaction_df['CohortMonth'] = grouping.transform('min')

# printing top 5 rows
print(transaction_df.head())

In [ ]:
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

# Getting the integers for date parts from the `InvoiceDay` column
transcation_year, transaction_month, _ = get_date_int(transaction_df, 'TransactionMonth')

# Getting the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, _ = get_date_int(transaction_df, 'CohortMonth')

In [ ]:
#  Get the  difference in years
years_diff = transcation_year - cohort_year

# Calculate difference in months
months_diff = transaction_month - cohort_month


""" Extract the difference in months from all previous values
 "+1" in addeded at the end so that first month is marked as 1 instead of 0 for easier interpretation. 
 """

transaction_df['CohortIndex'] = years_diff * 12 + months_diff  + 1 
print(transaction_df.head(5))

In [ ]:
# Counting daily active user from each chort
grouping = transaction_df.groupby(['CohortMonth', 'CohortIndex'])



# Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex
cohort_data = grouping['customer_id'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()


 # Assigning column names to the dataframe created above
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                 columns ='CohortIndex',
                                 values = 'customer_id')

# Printing top 5 rows of Dataframe
cohort_data.head()

In [ ]:
print(cohort_counts.round(1))

Calculate retention rate

In [ ]:
"""
Select the first column and store it to cohort_sizes as size of cohort === no. active customer in first day
We will store the 1st column as Cohort size i.e total Number of Customers in that Cohort
"""

cohort_sizes = cohort_counts.iloc[:,0]

"""
We will divide the values in other columns with Cohort Size in order to calculate the retention rate i.e Number of Customers in Each Cohort Index
"""

retention = cohort_counts.divide(cohort_sizes, axis=0)

In [ ]:
"""
this matrix has 2 axes: 
x - cohort date (beginning of month); y - cohort maturity (in months)
every cell is a unique count of customer for the x and y

if a customer's last order was in period 2,
they were active in period 0, 1 as well
same for other periods

"""

# Coverting the retention rate into percentage and Rounding off.
retention.round(3)*100

Visualizing the retention rate

In [ ]:
retention.index = retention.index.strftime('%Y-%m')
retention.index

In [ ]:
# Initialize the figure
plt.figure(figsize=(16, 10))

# Adding a title
plt.title('Retention Rate in percentage:- Monthly Cohorts', fontsize = 14)

# Creating the seaborn based heatmap
sns.heatmap(retention, annot=True, fmt= '.0%',cmap='YlGnBu', vmin = 0.0 , vmax = 0.6)
plt.ylabel('Cohort Month')
plt.xlabel('Cohort Index')
plt.yticks( rotation='360')
plt.show()

Analysis of retention rate

In [ ]:
# Create a groupby object and pass the monthly cohort and cohort index as a list
grouping = transaction_df.groupby(['CohortMonth', 'CohortIndex']) 

# Calculate the standard  average cost of the standard_cost column
cohort_data = grouping['standard_cost'].mean()

# Reset the index of cohort_data before pivot
cohort_data = cohort_data.reset_index()

# Create a pivot 
average_order = cohort_data.pivot(index='CohortMonth',
                                  columns='CohortIndex',
                                  values='standard_cost')

average_standard_cost = average_order.round(1)

In [ ]:
average_standard_cost.head()

In [ ]:
average_standard_cost.index = average_standard_cost.index.strftime('%Y-%m')
average_standard_cost.index

In [ ]:
# Initialize the figure
plt.figure(figsize=(16, 10))

# Adding a title
plt.title('Average Standard Cost: Monthly Cohorts', fontsize = 14)

# Creating the heatmap
sns.heatmap(average_standard_cost, annot = True,vmin = 0.0, vmax =20,cmap="YlGnBu", fmt='g')
plt.ylabel('Cohort Month')
plt.xlabel('Cohort Index')
plt.yticks( rotation='360')
plt.show()

Average list price monthly cohort

In [ ]:
# Create a groupby object and pass the monthly cohort and cohort index as a list
grouping = transaction_df.groupby(['CohortMonth', 'CohortIndex']) 

# Calculate the average of the list price column
cohort_data = grouping['list_price'].mean()

# Reset the index of cohort_data before pivot
cohort_data = cohort_data.reset_index()

# Create a pivot 
average_list = cohort_data.pivot(index='CohortMonth',
                                  columns='CohortIndex',
                                  values='list_price')

average_list_cost = average_list.round(1)

average_list_cost.head()

In [ ]:
average_list_cost.index = average_list_cost.index.strftime('%Y-%m')
average_list_cost.index

In [ ]:
# Initialize the figure
plt.figure(figsize=(16, 10))

# Adding a title
plt.title('Average List Price: Monthly Cohorts', fontsize = 14)

# Creating the heatmap
sns.heatmap(average_list_cost, annot = True,vmin = 0.0, vmax =20,cmap="nipy_spectral", fmt='g')
plt.ylabel('Cohort Month')
plt.xlabel('Cohort Index')
plt.yticks( rotation='360')
plt.show()